In [ ]:
!pip install rouge-score py7zr
!pip install transformers
!pip install datasets
!pip install tokenizers
!pip install evaluate
!pip install --upgrade accelerate

In [ ]:
!python -c "import nltk; nltk.download('punkt');"

In [ ]:
mkdir data

In [ ]:
import json
from typing import Any
from functools import partial

import datasets
from transformers import PreTrainedTokenizerBase
from nltk import sent_tokenize
from rouge_score import rouge_scorer
from transformers import AutoTokenizer

dataset = datasets.load_dataset('cnn_dailymail', name='3.0.0',split=["train[:10000]","validation[:1000]","test[:1000]"])
dataset = datasets.DatasetDict({'train':dataset[0],'validation':dataset[1],'test':dataset[2]})
src_text_column_name, tgt_text_column_name = "article", "highlights"
max_source_length, max_target_length = 1024, 128
n_proc = 40

# Since bos_token is used as the beginning of target sequence,
# we use mask_token to represent the beginning of each sentence.
bosent_token = "<mask>"
bosent_token_id = 50264

rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


def convert_to_features(
        examples: Any,
        tokenizer: PreTrainedTokenizerBase,
        padding: str,
        max_source_length: int,
        max_target_length: int,
        src_text_column_name: str,
        tgt_text_column_name: str,
):
    inputs, targets = [], []
    all_sent_rouge_scores = []
    for i in range(len(examples[src_text_column_name])):
        if examples[src_text_column_name][i] is not None and examples[tgt_text_column_name][i] is not None:
            input_sentences = sent_tokenize(examples[src_text_column_name][i])
            target_sentences = examples[tgt_text_column_name][i].strip()
            rouge_scores = []
            for sent in input_sentences:
                rouge_scores.append(rouge_scorer.score(target_sentences, sent)['rougeL'].fmeasure)
            # todo: add bos_token this way is unsafe
            inputs.append(bosent_token.join(input_sentences))
            targets.append(target_sentences.replace('\n', ' ').replace('  ', ' '))
            all_sent_rouge_scores.append(rouge_scores)
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # replace bos_token_id at the begining of document with bosent_token_id
    for i in range(len(model_inputs['input_ids'])):
        model_inputs['input_ids'][i][0] = bosent_token_id

    all_token_sent_id = []
    for sent_tokens in model_inputs['input_ids']:
        sid = -1
        token_sent_id = []
        for tid in sent_tokens:
            if tid == bosent_token_id:
                sid += 1
            if tid == tokenizer.eos_token_id or tid == tokenizer.pad_token_id:
                sid = -1
            token_sent_id.append(sid)
        all_token_sent_id.append(token_sent_id)

    all_token_info_dist = []
    all_sent_bos_idx = []
    for token_sent_id, sent_rouge_scores in zip(all_token_sent_id, all_sent_rouge_scores):
        sent_rouge_scores = sent_rouge_scores[:max(token_sent_id) + 1]  # truncation
        sent_bos_idx = []
        token_info_dist = []
        bos_idx = 0
        for sid in range(max(token_sent_id) + 1):
            tnum = token_sent_id.count(sid)
            sent_score = sent_rouge_scores[sid]
            token_info_dist.extend([sent_score for _ in range(tnum)])
            sent_bos_idx.extend([bos_idx for _ in range(tnum)])
            bos_idx += tnum
        token_info_dist.extend([-1 for _ in range(token_sent_id.count(-1))])
        all_token_info_dist.append(token_info_dist)
        sent_bos_idx.extend([0 for _ in range(token_sent_id.count(-1))])
        all_sent_bos_idx.append(sent_bos_idx)

    for i in range(len(all_token_sent_id)):
        for j in range(len(all_token_sent_id[i])):
            all_token_sent_id[i][j] += 1

    model_inputs['info_distribution'] = all_token_info_dist
    model_inputs['sentence_bos_index'] = all_sent_bos_idx
    model_inputs['sent_id'] = all_token_sent_id

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn", use_fast=False)

convert_to_features = partial(
    convert_to_features,
    tokenizer=tokenizer,
    padding='max_length',
    max_source_length=max_source_length,
    max_target_length=max_target_length,
    src_text_column_name=src_text_column_name,
    tgt_text_column_name=tgt_text_column_name,
)
dataset = dataset.map(
    convert_to_features,
    batched=True,
    num_proc=n_proc,
)

cols_to_keep = ["input_ids", "attention_mask", "labels", "info_distribution", "sentence_bos_index", "sent_id"]
dataset.set_format(columns=cols_to_keep)

for split in ['train', 'validation', 'test']:
    with open(f'data/{split}.json', 'w') as outfile:
        for i, example in enumerate(dataset[split]):
            json_string = json.dumps(example)
            outfile.write(json_string + '\n')


In [ ]:
!python run.py \
    --model_name_or_path facebook/bart-large-cnn \
    --do_train \
    --train_file data/train.json \
    --validation_file data/validation.json \
    --test_file data/test.json \
    --output_dir outputs/train \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 2 \
    --per_device_eval_batch_size 3 \
    --num_train_epochs 10 \
    --learning_rate 2e-5 \
    --warmup_steps 1500 \
    --weight_decay 0.01 \
    --max_grad_norm 0.1 \
    --metric_for_best_model rougeL \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --save_total_limit 2\
    --load_best_model_at_end True\
    --fp16 true \
    --bosent_token_id 50264 \
    --encoder_loss_ratio 1.0 \
    --encoder_label_smoothing 0.1 \
    --encoder_label_smoothing_type adjacent \
    --lower_saliency_threshold 0.125 \
    --higher_saliency_threshold 0.230 \
    --marginal_distribution true \
    --marginal_temperature 0.5 \
    --num_beams 5 \
    --max_length 128 \
    --min_length 20 \
    --length_penalty 1.5 \
    --no_repeat_ngram_size 3 \
    --overwrite_output_dir \
    --predict_with_generate


In [ ]:
!python run.py \
    --model_name_or_path /kaggle/working/outputs/season_cnndm \
    --tokenizer_name facebook/bart-large-cnn \
    --do_predict \
    --train_file data/train.json \
    --validation_file data/validation.json \
    --test_file data/test.json \
    --output_dir outputs/inference \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --num_beams 5 \
    --max_length 128 \
    --min_length 20 \
    --length_penalty 1.5 \
    --no_repeat_ngram_size 3 \
    --overwrite_output_dir \
    --predict_with_generate

2023-07-25 07:01:32.880456: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
07/25/2023 07:01:35 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2066: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(
[WARNING|logging.py:280] 2023-07-25 07:01:46,245 >> You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100% 33